In [5]:
import pandas as pd

In [6]:
data = pd.read_csv('../dataset/processed.csv',sep='/')

In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

c:\Users\KEY\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import faiss
import numpy as np
encoded_data = model.encode(data.text.tolist())
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(data))))
faiss.write_index(index, 'animes.index')

In [11]:
import time

def fetch_movie_info(dataframe_idx):
    info = data.iloc[dataframe_idx]
    meta_dict = dict()
    meta_dict['Name'] = info['title']
    return meta_dict
    
def search(query, top_k, index, model):
    t=time.time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    print('>>>> Results in Total Time: {}'.format(time.time()-t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_movie_info(idx) for idx in top_k_ids]
    return results

In [25]:
from pprint import pprint
query="anime that have action romance"
results=search(query, top_k=3, index=index, model=model)
print("\n")
for result in results:
    print('\t',result)

>>>> Results in Total Time: 1.036120891571045


	 {'Name': 'Junjou Romantica'}
	 {'Name': 'Sekirei'}
	 {'Name': 'Asagao to Kase-san.'}
